In [6]:
#%pip install python-dotenv
#%pip install openai
#%pip install langchain langchain_openai
#%pip install langchain-deepseek

### load ENV VARIABLE API KEY AND INSTANTIATE A CLIENT FOR THIS API

In [1]:
import os
COLAB = False
try:
    from google.colab import drive, userdata
    COLAB = True
    print("using colab")
except:
    print("not using colab")
#The deepseek-chat model points to DeepSeek-V3-0324 deepseek-chat
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
if COLAB:
    os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY') #or OPENAI_API_KEY
else:
    #OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

not using colab


### scrap some email corpus genAI from Jeff Heaton: https://www.youtube.com/watch?v=2VpOwFIGmA8

In [2]:
import requests

def get_email(i):
    if i<1 or i>10:
        raise Exception("invalid email number")
    
    url = f"https://data.heatonresearch.com/wustl/CABI/genai-langchain/emails/email_{i}.txt"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        corpus = response.text
        return corpus
    else:
        raise Exception("failed to retrieve the content")

In [5]:
print(get_email(2)) #genAI

Dear Professor,

I hope this message finds you well. I am currently working on Homework Assignment 7 and I'm finding some of the concepts challenging to grasp. Specifically, I'm struggling with the section on statistical analysis techniques. I've reviewed the lecture notes and the recommended textbook chapters, but I still have a few questions.

Could we possibly arrange a time to meet during your office hours this week? I believe a brief discussion would really help clarify my doubts. Thank you very much for your time and assistance.

Best regards,



### read these emails and classify them:
categories: spam, faculty (faculty announcements and requests), help (students requesting help on an assignment), lor (requesting letter of recommendation). If the email does not fit into these categories, say "other"

In [ ]:
from langchain_deepseek import ChatDeepSeek

MODEL = "deepseek-chat"
TEMPERATURE = 0.0
MAX_TOKENS = 1024

llm = ChatDeepSeek(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=None,
    max_retries=2,
    #stream=False,
    response_format={'type': 'text'} #json_object
)

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

email_context = """Classifiy the following email as either: 
          * spam - for marketing emails trying to sell something
          * faculty - for faculty announcements and requests
          * help - for students requesting help on an assignment
          * lor - for students requesting letters of recommendation
          * other - if it does not fit into any of these
          Here is the email. Return code, such as spam. Return nothing else, do not explain your choice.
          Make sure that if the email does not fit into one of these categories that you classify it as other.
          Here it is the email:
          {email}"""

help_context = """You are given an email where student is asking about an assignment. Return the assignment number that they are
               asking about. If you cannot tell return a ?. Return only the assignment number as integer, do not explain.
               Here it is the email:
               {email}
                """

email_prompt = PromptTemplate(input_variables=['email'], template=email_context)
help_prompt = PromptTemplate(input_variables=['email'], template=help_context)

chain_email = email_prompt | llm
chain_help = help_prompt | llm

for i in range(1,3):
    email = get_email(i)
    classification = chain_email.invoke(email).content.strip()
    if classification == 'help':
        assignment = chain_help.invoke(email).content.strip()
        print(f"Email #{i} is a question about assignment #{assignment}")
    else:
        print(f"Email #{i} is: {classification}")



c:\Users\josen\anaconda3\envs\tadsml\Lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\josen\anaconda3\envs\tadsml\Lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):


Email #1 is: lor
Email #2 is a question about assignment #7


### Here he have another application. Finding names within a text, that could be misclassified as typos. iamnotatypo.org

In [7]:
SAMPLE = """Esmae and Zarah have been friends since college. After graduation they inverviewed with priti for a job at the technotryne corporation, whose
flagship product is named Futuricore, which was developed by a programmer named María José García Martínez. After a successul interview they met with their friends Rafe and Ayda to
celebrate good times. Their first day on the job, they met three other employees, Ruaridh, Eesa, and Otillie. Later they were joined by Matei Smith"""

name_context = """Find all human names in the following text. Extract complete name, return only names.\n
                  Output each name on a separate line.\n
                  {subject}.\n"""

prompt = PromptTemplate(
    template=name_context,
    input_variables=['subject']
)

chain = prompt | llm

print(chain.invoke({'subject': SAMPLE}).content)

Here are the human names extracted from the text, each on a separate line:

Esmae  
Zarah  
priti  
María José García Martínez  
Rafe  
Ayda  
Ruaridh  
Eesa  
Otillie  
Matei Smith
